# Import the libraries

In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.utils import shuffle # Only used to shuffle the original input data
from scipy.special import logsumexp
from scipy.stats import multivariate_normal

# KMeans Algorithm

## KMeans Implementation Part

In [2]:
txt = np.loadtxt(r'./seeds_dataset.txt', dtype = float)
X = txt[:, :7]
y = txt[:, 7]

In [3]:
def compute_distance(x, y):
    return np.linalg.norm(y-x)

In [4]:
def initialize_centroid(data, K):
    data = list(data)
    init_centroid = random.sample(data, K)
    return init_centroid

In [5]:
def assign_centroids(curr_cluster, K):
    centroids = []
    for idx in range(K):
        centroid = np.mean(curr_cluster[idx], axis = 0)
        centroids.append(centroid)
    return centroids

In [6]:
def assign_cluster(data, curr_centroids, K):
    curr_cluster = {}
    for i in range(K):
        curr_cluster[i] = []
    for i in data:
        clusteridx = -1
        dist = []
        for j in range(K):
            dist.append(compute_distance(i, curr_centroids[j]))
        clusteridx = dist.index(min(dist))
        curr_cluster[clusteridx].append(i)
    return curr_cluster

In [7]:
def fit_kmeans(k, tolerence):
    K = k
    tol = tolerence
    centroids = initialize_centroid(X, K)
    current_clusters = assign_cluster(X, centroids, K)
    loop = True
    while loop:
        prev = centroids
        centroids = assign_centroids(current_clusters, K)
        current_clusters = assign_cluster(X, centroids, K)
        dist_sum = 0
        for i in range(len(centroids)):
            dist_sum += compute_distance(prev[i], centroids[i])
        if dist_sum < tol:
            loop = False
    return current_clusters

# GMM Algorithm

## GMM Implementation Part

In [8]:
def initialize_para(data, K):
    pi = [1 / K] * K
    split_data = np.array_split(data, K)
    mu = [np.mean(split, axis = 0) for split in split_data]
    sigma = [np.cov(split.T) for split in split_data]
    return pi, mu, sigma

In [9]:
def update_gamma(data, K, pi, mu, sigma):
    gamma = np.zeros((len(data), K))
    for n in range(len(data)):
        for k in range(K):
            num = pi[k] * multivariate_normal.pdf(data[n], mu[k], sigma[k])
            deno = []
            for j in range(K):
                element = pi[j] * multivariate_normal.pdf(data[n], mu[j], sigma[j])
                deno.append(element)
            gamma[n][k] = num / logsumexp(deno)
    return gamma

In [10]:
def update_mu(data, K, gamma, N, mu):
    for k in range(K):
        musum = 0
        for n in range(len(data)):
            musum += gamma[n][k] * data[n]
        mu[k] = musum / N[k]
    return mu

In [11]:
def update_sigma(data, K, mu, gamma, sigma, N):
    for k in range(K):
        musum = 0
        for n in range(len(data)):
            musum += gamma[n][k] * np.outer((data[n] - mu[k]), (data[n] - mu[k]).T)
        sigma[k] = musum / N[k]
    return sigma

In [12]:
def predict(data, K, pi, mu, sigma):
    gamma = update_gamma(data, K, pi, mu, sigma)
    label = []
    for i in range(len(data)):
        l = np.argmax(gamma[i])
        label.append(l)
    return label

In [13]:
def fit_GMM(data, K, pi, mu, sigma, tol):
    flag = True
    while flag:
        gamma = update_gamma(data, K, pi, mu, sigma)
        N = np.sum(gamma, axis = 0)
        mu = update_mu(data, K, gamma, N, mu)
        sigma = update_sigma(data, K, mu, gamma, sigma, N)
        prev = list(pi)
        pi = [N[k] / len(data) for k in range(K)]
        diffsum = 0
        for i in range(len(prev)):
            diffsum += (prev[i] - pi[i])**2
        if diffsum <= tol:
            flag = False
    return pi, mu, sigma

In [14]:
def get_curr_clusters(label, data, K):
    labelarr = np.array(label)
    labelarr = labelarr[:,np.newaxis]
    new = np.hstack((data, labelarr))
    current_clusters = {}
    for i in range(K):
        current_clusters[i] = []
    for i in range(len(data)):
        if new[i, 7] == 0:
            current_clusters[0].append(data[i, :7])
        elif new[i, 7] == 1:
            current_clusters[1].append(data[i, :7])
        elif new[i, 7] == 2:
            current_clusters[2].append(data[i, :7])
    return current_clusters

# Calculate Silhouette Coefficient

In [15]:
def silhouette_coefficient(current_clusters, K):
    coeflist = []
    for i in range(K):
        near_num = [m for m in range(K)]
        near_num.remove(i)
        for j in current_clusters[i]:
            a = 0
            for k in current_clusters[i]:
                if np.array_equal(j, k) == False:
                    a += compute_distance(j, k)
            a = a / (len(current_clusters[i])-1)
            b1 = []
            for m in near_num:
                blst = []
                for n in current_clusters[m]:
                    blst.append(compute_distance(j, n))
                bsum = sum(blst)/len(blst)
                b1.append(bsum)
            b = min(b1)
            s = (b - a) / max(a, b)
            coeflist.append(s)
    return sum(coeflist)/210

# Calculate Rand Index

In [16]:
def rand_index(current_clusters, txt, K):
    predictlist = []
    for i in range(len(current_clusters)):
        for j in current_clusters[i]:
            j = np.insert(j, 7, i)
            predictlist.append(j)
    matrix = np.stack(predictlist, axis = 0)
    r1 = np.core.records.fromarrays([matrix[:,1],matrix[:,0]],names='a,b')
    matrix = matrix[r1.argsort()]
    r2 = np.core.records.fromarrays([txt[:,1],txt[:,0]],names='a,b')
    txt_new = txt[r2.argsort()]
    a = 0
    b = 0
    c = 0
    d = 0
    for i in range(len(matrix)):
        for j in range(i+1, len(matrix)):
            if (matrix[i, 7] == matrix[j, 7]) & (txt_new[i, 7] == txt_new[j, 7]):
                a += 1
            elif (matrix[i, 7] != matrix[j, 7]) & (txt_new[i, 7] != txt_new[j, 7]):
                b += 1
            elif (matrix[i, 7] == matrix[j, 7]) & (txt_new[i, 7] != txt_new[j, 7]):
                c += 1
            elif (matrix[i, 7] != matrix[j, 7]) & (txt_new[i, 7] == txt_new[j, 7]):
                d += 1
    RI = (a + b) / (a + b + c + d)
    return RI

# Calculate the sensitivity to the initialization

In [17]:
def performance_evaluation_Kmeans():
    result = pd.DataFrame(columns = ['silhouette_coef', 'rand_index'])
    for i in range(10):
        curr_cluster = fit_kmeans(3, 0.000001)
        sil = silhouette_coefficient(curr_cluster, 3)
        ran = rand_index(curr_cluster, txt, 3)
        chart = [sil, ran]
        result.loc[len(result)] = chart
    return result

In [18]:
def performance_evaluation_GMM():
    result = pd.DataFrame(columns = ['silhouette_coef', 'rand_index'])
    for i in range(10):
        X_ = shuffle(X)
        pi, mu, sigma = initialize_para(X_, 3)
        pi, mu, sigma = fit_GMM(X_, 3, pi, mu, sigma, 0.000001)
        label = predict(X_, 3, pi, mu, sigma)
        curr_cluster = get_curr_clusters(label, X_, 3)
        sil = silhouette_coefficient(curr_cluster, 3)
        ran = rand_index(curr_cluster, txt, 3)
        chart = [sil, ran]
        result.loc[len(result)] = chart
    return result

# Silhouette Coefficient and Rand Index for KMeans

In [19]:
curr_cluster = fit_kmeans(3, 0.000001)
sil = silhouette_coefficient(curr_cluster, 3)
ran = rand_index(curr_cluster, txt, 3)

In [20]:
sil

0.47193373191268945

In [21]:
ran

0.8743677375256322

In [22]:
res = performance_evaluation_Kmeans()
sil_std = res['silhouette_coef'].std()
ran_std = res['rand_index'].std()

In [23]:
sil_std

0.001832990974720474

In [24]:
ran_std

0.0014527695986154717

# Silhouette Coefficient and Rand Index for GMM

In [25]:
X_ = shuffle(X)
pi, mu, sigma = initialize_para(X_, 3)
pi, mu, sigma = fit_GMM(X_, 3, pi, mu, sigma, 0.000001)
label = predict(X_, 3, pi, mu, sigma)
current_clusters = get_curr_clusters(label, X_, 3)
sil = silhouette_coefficient(current_clusters, 3)
ran = rand_index(current_clusters, txt, 3)

In [26]:
sil

0.4442191107942514

In [27]:
ran

0.8769195716564138

In [28]:
result = performance_evaluation_GMM()
sil_std = result['silhouette_coef'].std()
ran_std = result['rand_index'].std()

In [29]:
sil_std

0.03991633229536218

In [30]:
ran_std

0.0602683539129245